In [1]:
from datasets import load_dataset

# D:\\datasets
wikipedia = load_dataset(
    "wikipedia", date="20240320", language="en", cache_dir="/mnt/d/datasets", num_proc=20
)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting content from /mnt/d/datasets/downloads/857dfbe3f55bd378c379aaa2fd24a2f96c05747fbce0713c9d17f28485c0dd95
Extracting content fromExtracting content from  Extracting content from/mnt/d/datasets/downloads/c7656a812ca7e984fba82f35cbcb3d0587815b152ab1e51b5dff3f500c24f128/mnt/d/datasets/downloads/d96155d173be521ce90e436f10a502a46711bb08871aceac68241cfbe91ed2ceExtracting content from
  Extracting content fromExtracting content from/mnt/d/datasets/downloads/5a6cb88e39d1c88284aee5d60423aaf2fb63fa1c0591971e43e76250168ffdeb /mnt/d/datasets/downloads/8e44c7fe1fe5cc83c43e0c744c3bb3ee84dbc45438812f903a8e2b5f06d7698d
/mnt/d/datasets/downloads/c7ecc2d088152936964fee96e9bc3e3ef73b836ac64f2609932e391d5df20faa 

/mnt/d/datasets/downloads/d0af3f9c059f67ec29738095a5d9846afa833535048878ce0a99a075d53925bd

Extracting content from /mnt/d/datasets/downloads/3cf6627528185b053e57843ca731130ac73bd87c9a3d5c16d9f94c753503af21Extracting content from
 /mnt/d/datasets/downloads/ee3036f4b1dc806278d42dea1b6b0a

In [44]:

base_wikipedia_url = "https://en.wikipedia.org"

# /mnt/c/Users/andre/OneDrive/Documents/coding projects/chunk_wikipedia/scrape_vital_articles/vital_article_links.txt
# scrape_vital_articles/vital_article_links.txt
with open("scrape_vital_articles/vital_article_links.txt", "r") as f:
    vital_article_urls_file = f.readlines()

vital_article_urls = set([
    base_wikipedia_url + url.strip().replace("_", "%20").replace("(", "%28").replace(")", "%29").replace(",", "%2C").replace("'", "%27").replace('"', '%22').replace("!", "%21").replace(":", "%3A")  for url in vital_article_urls_file
])

# .replace('"', '%22').replace("-", "")

In [45]:
filtered_wikipedia = wikipedia.filter(lambda x: x["url"] in vital_article_urls, num_proc=20)

print(filtered_wikipedia)

Filter (num_proc=20):   0%|          | 0/6797834 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 49917
    })
})


In [46]:
dataset_urls = set(wikipedia['train']["url"])

In [47]:
# what urls are in vital_article_urls that are not in dataset_urls
missing_urls = vital_article_urls - dataset_urls
print(len(missing_urls))
missing_urls


33


{'https://en.wikipedia.org#Engineering',
 'https://en.wikipedia.org#Machinery%20and%20tools',
 'https://en.wikipedia.org#Military%20technology',
 'https://en.wikipedia.org#Military%20technology%20%28303%20articles%29',
 'https://en.wikipedia.org/wiki/Barbara%20H.%20Partee',
 'https://en.wikipedia.org/wiki/Donkey%20Kong%20%281981%20video%20game%29',
 'https://en.wikipedia.org/wiki/Ecological%20restoration',
 'https://en.wikipedia.org/wiki/Flavius%20Aetius%20%28magister%20militum%29',
 'https://en.wikipedia.org/wiki/History%20of%20Agriculture',
 'https://en.wikipedia.org/wiki/History%20of%20atomic%20theory',
 'https://en.wikipedia.org/wiki/Internal%20condom',
 'https://en.wikipedia.org/wiki/Kim%20Yo-jong',
 'https://en.wikipedia.org/wiki/Lead-acid%20battery',
 'https://en.wikipedia.org/wiki/M*A*S*H%20%28TV%20series%29',
 'https://en.wikipedia.org/wiki/Molten-salt%20reactor',
 'https://en.wikipedia.org/wiki/Mumtaz%20%28Indian%20actress%29',
 'https://en.wikipedia.org/wiki/Na%20Hye-s%C5%8F

In [67]:
# url.replace("-", "%20") 1
# url.replace("-", "%E2%80%93") 1

new_missing_urls = set([url.replace("-", "%E2%80%93") for url in missing_urls])

additional_entries = wikipedia.filter(lambda x: x["url"] in new_missing_urls, num_proc=8)
additional_entries

Filter (num_proc=8):   0%|          | 0/6797834 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 1
    })
})

In [68]:
from datasets import concatenate_datasets

filtered_wikipedia = concatenate_datasets([filtered_wikipedia, additional_entries['train']])

In [59]:
from datasets import concatenate_datasets

filtered_wikipedia = concatenate_datasets([filtered_wikipedia['train'], additional_entries])

ValueError: Expected a list of Dataset objects or a list of IterableDataset objects, but element at position 0 is a dict.

In [52]:
print(additional_entries['train'][1])

{'id': '840191', 'url': 'https://en.wikipedia.org/wiki/Sagittarius%20A%2A', 'title': 'Sagittarius A*', 'text': 'Sagittarius A* ( ), abbreviated Sgr\xa0A* ( ), is the supermassive black hole at the Galactic Center of the Milky Way. Viewed from Earth, it is located near the border of the constellations Sagittarius and Scorpius, about 5.6° south of the ecliptic, visually close to the Butterfly Cluster (M6) and Lambda Scorpii.\n\nThe object is a bright and very compact astronomical radio source. The name Sagittarius A* distinguishes the compact source from the larger (and much brighter) Sagittarius\xa0A (Sgr\xa0A) region in which it is embedded.  Sgr\xa0A* was discovered in 1974 by  and Robert L. Brown, and the asterisk  was assigned in 1982 by Brown, who understood that the strongest radio emission from the center of the galaxy appeared to be due to a compact nonthermal radio object.\n\nThe observations of several stars orbiting Sagittarius A*, particularly star S2, have been used to dete

In [55]:
data = wikipedia.filter(lambda x: 'https://en.wikipedia.org/wiki/History%20of%20agriculture' == x['url'], num_proc=8)
data

Filter (num_proc=8):   0%|          | 0/6797834 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 1
    })
})

In [62]:
from datasets import concatenate_datasets

filtered_wikipedia = concatenate_datasets([filtered_wikipedia['train'], additional_entries['train'], data['train']])

In [54]:
for d in data['train']:
    print(d['url']) 

https://en.wikipedia.org/wiki/History%20of%20agriculture%20in%20the%20United%20States
https://en.wikipedia.org/wiki/History%20of%20agriculture
https://en.wikipedia.org/wiki/History%20of%20agriculture%20in%20China
https://en.wikipedia.org/wiki/History%20of%20agriculture%20in%20the%20Indian%20subcontinent
https://en.wikipedia.org/wiki/History%20of%20agriculture%20in%20Cheshire
https://en.wikipedia.org/wiki/History%20of%20agriculture%20in%20Scotland
https://en.wikipedia.org/wiki/History%20of%20agriculture%20in%20Canada
https://en.wikipedia.org/wiki/History%20of%20agriculture%20in%20Chile
https://en.wikipedia.org/wiki/History%20of%20agriculture%20in%20Argentina
https://en.wikipedia.org/wiki/History%20of%20agriculture%20in%20Palestine


In [70]:
filtered_wikipedia.push_to_hub("vital_wikipedia_articles")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/50 [00:00<?, ?ba/s]

In [71]:
wikipedia.push_to_hub("wikipedia_20240320")

Uploading the dataset shards:   0%|          | 0/45 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/152 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/152 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/152 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/152 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/152 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/152 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/152 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/152 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/140 [00:00<?, ?B/s]